# FP-Tree 

目的：克服 Apriori 算法在复杂度和效率方面的缺陷
(Apriori算法在挖掘频繁模式时，需要多次扫描数据库，并且会产生大量的候选项集。)

2000 年，Han Jiawei 等人提出了基于频繁模式树（Frequent Pattern Tree, FP—Tree）的发现频繁模式的算法 FP-Growth。其思想是构造一棵 FP-Tree，把数据集中的数据映射到树上，再根据这棵 FP-Tree 找出所有频繁项集。

優勢：FP-Growth算法在进行频繁模式挖掘时，只需要对数据库进行两次扫描，并且不会产生候选项集。它的效率相比于Apriori算法有很大的提高。

參考來源：https://zhuanlan.zhihu.com/p/139938058 、 https://zhuanlan.zhihu.com/p/444420809

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
import time

# Load your data
file_path = 'grouped_data.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Splitting the Description column into individual items
data['Items'] = data['Description'].str.split(',')


# Start the timer for FP-growth algorithm
start_time = time.time()

# Extracting the list of transactions
transactions = data['Items'].tolist()

# TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# FP-growth 
frequent_itemsets = fpgrowth(df, min_support=0.01, use_colnames=True)

# Generating the association rules
FP_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)


# Stop the timer and print the time taken for FP-growth
fp_growth_time = time.time() - start_time
print(f"Time taken for FP-growth: {fp_growth_time} seconds")

Time taken for FP-growth: 3.700408935546875 seconds


In [3]:
# Calculating confidence and lift
# These metrics are already included in the rules DataFrame
FP_rules = FP_rules[['antecedents', 'consequents', 'confidence', 'lift']]
FP_rules

,antecedents,consequents,confidence,lift
0,( POPPY'S PLAYHOUSE KITCHEN),( POPPY'S PLAYHOUSE BEDROOM ),0.706977,41.645189
1,( POPPY'S PLAYHOUSE BEDROOM ),( POPPY'S PLAYHOUSE KITCHEN),0.732530,41.645189
2,( ALARM CLOCK BAKELIKE GREEN),( ALARM CLOCK BAKELIKE RED ),0.632184,14.960666
3,( RED HANGING HEART T-LIGHT HOLDER),( WHITE HANGING HEART T-LIGHT HOLDER),0.626074,7.298530
4,( JUMBO STORAGE BAG SUKI),( JUMBO BAG RED RETROSPOT),0.601724,7.235489
...,...,...,...,...
188,( REGENCY TEA PLATE GREEN ),( REGENCY TEA PLATE PINK),0.719251,57.648584
189,( REGENCY TEA PLATE PINK),( REGENCY TEA PLATE ROSES ),0.839344,48.052950
190,( GARDENERS KNEELING PAD CUP OF TEA ),( GARDENERS KNEELING PAD KEEP CALM ),0.697706,19.426100
191,( JUMBO BAG VINTAGE CHRISTMAS ),( JUMBO BAG 50'S CHRISTMAS ),0.657143,23.799280


In [4]:
# 對規則按confidence進行降序排序
sorted_rules_by_confidence = FP_rules.sort_values(by='confidence', ascending=False)

# 選取前幾條規則
top_10_rules_by_confidence = sorted_rules_by_confidence.head(5)
top_10_rules_by_confidence

,antecedents,consequents,confidence,lift
41,( AIRLINE LOUNGE),(METAL SIGN),1.0,83.433447
133,( SET 3 RETROSPOT TEA),"(COFFEE, SUGAR)",1.0,55.559091
123,(SUGAR),(COFFEE),1.0,45.523277
125,( SET 3 RETROSPOT TEA),(SUGAR),1.0,55.559091
127,( SET 3 RETROSPOT TEA),(COFFEE),1.0,45.523277


In [13]:
# 對規則按lift進行降序排序
sorted_rules_by_confidence = FP_rules.sort_values(by='lift', ascending=False)

# 選取前幾條規則
top_10_rules_by_confidence = sorted_rules_by_confidence.head(10)
top_10_rules_by_confidence

,antecedents,consequents,confidence,lift
41,( AIRLINE LOUNGE),(METAL SIGN),1.000000,83.433447
42,(METAL SIGN),( AIRLINE LOUNGE),0.986348,83.433447
88,( RETRO SPOT),( BIRTHDAY CARD),0.985185,58.455915
89,( BIRTHDAY CARD),( RETRO SPOT),0.645631,58.455915
188,( REGENCY TEA PLATE GREEN ),( REGENCY TEA PLATE PINK),0.719251,57.648584
187,( REGENCY TEA PLATE PINK),( REGENCY TEA PLATE GREEN ),0.881967,57.648584
124,(SUGAR),( SET 3 RETROSPOT TEA),0.961364,55.559091
128,"(COFFEE, SUGAR)",( SET 3 RETROSPOT TEA),0.961364,55.559091
129,"(COFFEE, SET 3 RETROSPOT TEA)",(SUGAR),1.000000,55.559091
132,(SUGAR),"(COFFEE, SET 3 RETROSPOT TEA)",0.961364,55.559091


# apriori

In [19]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Load your data
file_path = 'grouped_data.csv'
data = pd.read_csv(file_path)

# Start the timer for apriori algorithm
start_time = time.time()

# 將描述列表轉換為交易列表
transactions = data['Description'].apply(lambda x: x.split(', ')).tolist()
#print(transactions)

# 初始化交易編碼器
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# 將交易數據轉換為適合進行Apriori算法的DataFrame格式
df = pd.DataFrame(te_ary, columns=te.columns_)


# 使用Apriori算法找出頻繁項集，設定最小支持度為0.01
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

# 使用關聯規則函數生成規則，設定最小置信度為0.6
apriori_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.6)


# Stop the timer and print the time taken for apriori
apriori_time = time.time() - start_time
print(f"Time taken for apriori: {apriori_time} seconds")

Time taken for apriori: 51.58944010734558 seconds


In [20]:
# Calculating confidence and lift
# These metrics are already included in the rules DataFrame
apriori_rules = apriori_rules[['antecedents', 'consequents', 'confidence', 'lift']]
apriori_rules

,antecedents,consequents,confidence,lift
0,(),(BIRTHDAY CARD),0.466205,27.396239
1,(BIRTHDAY CARD),(),0.646635,27.396239
2,(),(FANCY FONT BIRTHDAY CARD),0.561525,42.367418
3,(FANCY FONT BIRTHDAY CARD),(),1.000000,42.367418
4,(6 RIBBONS RUSTIC CHARM),(WHITE HANGING HEART T-LIGHT HOLDER),0.267430,2.839960
...,...,...,...,...
1477,"(JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAG...","(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SUKI)",0.393851,18.587014
1478,(JUMBO STORAGE BAG SUKI),"(JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROS...",0.223980,14.408988
1479,(JUMBO BAG PINK POLKADOT),"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO STOR...",0.218522,13.662346
1480,(JUMBO BAG RED RETROSPOT),"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SU...",0.125995,9.935747


In [21]:
# 對規則按confidence進行降序排序
sorted_rules_by_confidence = apriori_rules.sort_values(by='confidence', ascending=False)

# 選取前10條規則
top_10_rules_by_confidence = sorted_rules_by_confidence.head(5)
top_10_rules_by_confidence

,antecedents,consequents,confidence,lift
56,(BACK DOOR ),(KEY FOB ),1.000000,46.037665
3,(FANCY FONT BIRTHDAY CARD),(),1.000000,42.367418
67,(RETRO SPOT),(BIRTHDAY CARD),1.000000,58.764423
556,(SHED),(KEY FOB ),1.000000,46.037665
1402,"(REGENCY TEA PLATE PINK, REGENCY TEA PLATE ROS...",(REGENCY TEA PLATE GREEN ),0.948529,60.071891


In [22]:
# 對規則按confidence進行降序排序
sorted_rules_by_confidence = apriori_rules.sort_values(by='lift', ascending=False)

# 選取前10條規則
top_10_rules_by_confidence = sorted_rules_by_confidence.head(10)
top_10_rules_by_confidence

,antecedents,consequents,confidence,lift
1404,(REGENCY TEA PLATE PINK),"(REGENCY TEA PLATE GREEN , REGENCY TEA PLATE R...",0.821656,62.379515
1401,"(REGENCY TEA PLATE GREEN , REGENCY TEA PLATE R...",(REGENCY TEA PLATE PINK),0.801242,62.379515
1402,"(REGENCY TEA PLATE PINK, REGENCY TEA PLATE ROS...",(REGENCY TEA PLATE GREEN ),0.948529,60.071891
1403,(REGENCY TEA PLATE GREEN ),"(REGENCY TEA PLATE PINK, REGENCY TEA PLATE ROS...",0.668394,60.071891
66,(BIRTHDAY CARD),(RETRO SPOT),0.649038,58.764423
67,(RETRO SPOT),(BIRTHDAY CARD),1.000000,58.764423
815,(REGENCY TEA PLATE PINK),(REGENCY TEA PLATE GREEN ),0.898089,56.877430
814,(REGENCY TEA PLATE GREEN ),(REGENCY TEA PLATE PINK),0.730570,56.877430
812,(REGENCY MILK JUG PINK ),(REGENCY SUGAR BOWL GREEN),0.720588,51.963127
813,(REGENCY SUGAR BOWL GREEN),(REGENCY MILK JUG PINK ),0.722714,51.963127


In [18]:
top_10_rules_by_confidence.to_csv("Apriori ALL.csv")

# FP-Tree ,  apriori 計算時間比較

Time taken for apriori: 39.07537817955017 seconds

Time taken for FP-growth: 2.373975992202759 seconds